In [5]:
# select * from table where text like '%internet%'

# What’s the internet usage policy?

# Step 1: Define Sample Documents
documents = [
    {"doc_id": "1", "section": "Pay Policies", "content": "Employees are paid bi-weekly via direct deposit."},
    {"section": "Leave of Absence", "content": "Employees must submit a leave request for approval."},
    {"section": "Internet Use", "content": "Company internet must be used for work-related tasks only."},
    {"section": "Internet Use", "content": "Company internet is a broadband internet."},
    {"section": "Break at Work", "content": "Employees can take an hour break."},
    {"section": "Harassment", "content": "Interact with each employee with Respect"}
]

# "What’s the internet usage policy?"

# No text comparison - Only Vector/ Embeddings comparison

# ([0.12655601, 0.09747529, 0.44755298, 0.11316206, 0.04429515],

# 0.12655601 - Query Vector is 12% similar to document 1's vector - Query is 12% similar to document 1

# Step 2: Get Content Texts
content_corpus = [doc["content"] for doc in documents]
content_corpus

['Employees are paid bi-weekly via direct deposit.',
 'Employees must submit a leave request for approval.',
 'Company internet must be used for work-related tasks only.',
 'Company internet is a broadband internet.',
 'Employees can take an hour break.',
 'Interact with each employee with Respect']

In [6]:
%pip install -q sentence-transformers


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
doc_vectors = model.encode(content_corpus)

/Users/suma/Training/GenAI_RAG_Eval/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
doc_vectors

array([[ 0.02472514, -0.00908146,  0.0388713 , ...,  0.0196564 ,
         0.04260007, -0.02707142],
       [ 0.03315507,  0.04853379,  0.04736274, ...,  0.10182011,
         0.0915928 ,  0.00358368],
       [-0.07135908, -0.03066469,  0.03183772, ..., -0.04109802,
         0.06524781, -0.00688534],
       [-0.0038374 , -0.0233675 ,  0.0295868 , ..., -0.04415294,
         0.1255909 , -0.03139856],
       [-0.01790445,  0.01495852,  0.08163831, ..., -0.03217232,
        -0.0051365 ,  0.05279535],
       [-0.00240885,  0.03361142, -0.06162645, ...,  0.04830882,
         0.0370764 , -0.01683048]], shape=(6, 384), dtype=float32)

In [9]:
# Step 3: User Query and Semantic Matching
import numpy as np

query = "What’s the internet usage policy?"
query_vec = model.encode([query])[0]
query_vec

array([ 2.39816820e-03, -4.11839932e-02, -2.52490565e-02, -4.67050076e-02,
        4.32103314e-03,  1.65876430e-02,  1.20891653e-01, -3.50352898e-02,
        2.16827309e-03, -1.62890152e-04,  2.62875985e-02,  9.05028209e-02,
       -2.66066510e-02, -1.82131808e-02,  3.06277778e-02,  1.67854987e-02,
        1.55614102e-02, -8.26497748e-02, -3.40456404e-02, -3.08671743e-02,
        7.89995566e-02, -3.16904187e-02,  1.35831395e-02,  9.12274292e-04,
       -1.05809318e-02,  3.91190648e-02, -3.48707736e-02,  8.64355243e-05,
       -3.52702476e-02,  3.56902331e-02,  9.55271535e-03, -3.57899517e-02,
        4.84947534e-03, -4.10227180e-02, -7.66861811e-02, -1.00646734e-01,
       -9.23561081e-02, -2.47275387e-03, -2.74321344e-02,  2.85045840e-02,
        2.86296494e-02, -7.78359100e-02, -2.46462156e-03,  9.98250544e-02,
        5.86107075e-02,  2.24836506e-02,  1.56647016e-04,  1.44218225e-02,
        5.55950392e-04,  3.22429426e-02,  1.03300072e-01,  3.42920125e-02,
        3.94683145e-02,  

In [10]:
similarities = model.similarity(query_vec, doc_vectors)

# Ensure it's a 1D numpy array
similarities = np.asarray(similarities).squeeze()
similarities

array([0.12655601, 0.09747529, 0.44755298, 0.4798251 , 0.11316206,
       0.04429515], dtype=float32)

In [11]:
# Now get top 3
top_3_indices = np.argsort(similarities)[::-1][:3]
print(top_3_indices)
top_scores = similarities[top_3_indices]
top_scores

[3 2 0]


array([0.4798251 , 0.44755298, 0.12655601], dtype=float32)

In [12]:
top_docs = [documents[i]['content'] for i in top_3_indices]
# documents = [
#     {"section": "Pay Policies", "content": "Employees are paid bi-weekly via direct deposit."},
#     {"section": "Leave of Absence", "content": "Employees must submit a leave request for approval."},
#     {"section": "Internet Use", "content": "Company internet must be used for work-related tasks only."},
#     {"section": "Break at Work", "content": "Employees can take an hour break."},
#     {"section": "Harassment", "content": "Interact with each employee with Respect"}
# ]

print (top_docs)
context = ",".join(top_docs)
context

['Company internet is a broadband internet.', 'Company internet must be used for work-related tasks only.', 'Employees are paid bi-weekly via direct deposit.']


'Company internet is a broadband internet.,Company internet must be used for work-related tasks only.,Employees are paid bi-weekly via direct deposit.'

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True, dotenv_path="../.env")
my_api_key = os.getenv("OPENAI_API_KEY")

my_client = OpenAI(api_key=my_api_key)
# my_client

def ask_question_open_ai(prompt):

    # print(f"User asked: {prompt}")
    # my_client.chat.completions.create

    llm_response = my_client.chat.completions.create(
        model="gpt-5-nano",
        # messages=[
        #     {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
        #     {"role": "user", "content": prompt}
        # ]
        messages=[
            {"role": "system", "content": '''
             You are an assistant who answers only based on the given context.
             '''},
            {"role": "user", "content": f"Context: {context}\n\n User Question: {query}"} 
        ]

    )
    return llm_response.choices[0].message.content  

In [14]:
print (query)
response = ask_question_open_ai(query)

What’s the internet usage policy?


In [15]:
print(f"User query: {query}")
print(f"Context: {context}")

print(f"\n\nOpen AI Response: {response}")

User query: What’s the internet usage policy?
Context: Company internet is a broadband internet.,Company internet must be used for work-related tasks only.,Employees are paid bi-weekly via direct deposit.


Open AI Response: The internet usage policy is that the company internet (which is broadband) must be used only for work-related tasks.
